In [1]:
import pandas as pd
from tqdm import tqdm
import openpyxl
import os
import json
import geopandas as gpd

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Note: only load the shapefile dataframe when needed to merge. This will inflate the filesize of the generated csv

# shp_path = os.path.join('..', 'philippines-psgc-shapefiles', 'philippines-psgc-shapefiles', 'data', '2023', 'Municities', 'phl_admbnda_adm3_psa_namria_20231106.shp')
# shape_file_df = gpd.read_file(shp_path)
# map_df = shape_file_df[['ADM3_EN', 'ADM2_EN', 'ADM1_EN', 'geometry']]

In [2]:
# Defining Constants

SAVE_FILE_TYPE = 'csv'
YEARS = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

# Processing Summary Statistics

In [3]:
%%time

def preprocess_port_data(file_path):

    data = pd.read_excel(file_path, header=[0, 1, 2, 3])
    
    new_columns = []
    for col in data.columns:
        col_names = [str(c).strip() for c in col if 'Unnamed' not in str(c)]
        new_col_name = ' | '.join(col_names) if col_names else 'Unknown'
        new_columns.append(new_col_name)
    
    data.columns = new_columns
    data.dropna(how='all', axis=1, inplace=True)
    data = data.loc[~(data.map(lambda x: pd.isna(x) or str(x).strip() == '')).all(axis=1)]
    
    data.dropna(how='all', axis=0, inplace=True)
    data.reset_index(drop=True, inplace=True)

    # Hardcode renaming the first three columns to 'Category 1', 'Category 2', 'Category 3'
    data.columns.values[0] = 'Category 1'
    data.columns.values[1] = 'Category 2'
    data.columns.values[2] = 'Category 3'

    # Add the quarter to the 'TOTAL' column names correctly
    quarters = ['1st Quarter', '2nd Quarter', '3rd Quarter', '4th Quarter']
    for quarter in quarters:
        for i, col in enumerate(data.columns):
            if col == 'TOTAL' and i > 0 and quarter in data.columns[i+1]:
                data.columns.values[i] = f'{quarter} TOTAL'

    # Fix 'GRAND TOTAL' columns
    data.columns = [col.replace('GRAND TOTAL | ', 'GRAND TOTAL | ') for col in data.columns]
    data.columns = [col.replace('Unknown', '') for col in data.columns]

    # Remove duplicate rows
    data.drop_duplicates(inplace=True)
    data['Category 1'].ffill(inplace=True)

    return data

for year in tqdm(YEARS, desc='Processing Summary Statistics'):
    file_path = os.path.join('..', '[DATA] Ports Files', 'Raw Files', year, f'{year}SummaryStatistics.xlsx')
    save_path = os.path.join('..', '[DATA] Ports Files', 'Standardized', 'Summary Statistics', f'{year}SummaryStatistics.{SAVE_FILE_TYPE}')
    cleaned_data = preprocess_port_data(file_path)
    cleaned_data.to_csv(save_path, index=False)

print('Success!')

Processing Summary Statistics: 100%|███████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.56it/s]

Success!
CPU times: total: 2.09 s
Wall time: 2.26 s


# Processing Shipping Data

In [4]:
%%time

def map_city(pmo, pier_terminal):
    """Map the city based on PMO and Pier/Terminal using the provided mapping."""
    mapping_file = os.path.join('..', 'port_city_mapper.json')
    with open(mapping_file, 'r') as f:
        port_city_mapper = json.load(f)
    
    if pd.isna(pmo) or pd.isna(pier_terminal):
        return pier_terminal
    
    pmo = pmo.upper()
    pier_terminal = pier_terminal.upper()
    
    if pmo in port_city_mapper:
        for key in port_city_mapper[pmo]:
            if key in pier_terminal:
                return port_city_mapper[pmo][key]
    
    return pier_terminal

def merge_with_shapefile(shipping_df, map_df):
    """Merge the shipping data with the PSGC shapefile data."""
    mapping_file = os.path.join('..', 'ambiguos_cities.json')
    with open(mapping_file, 'r') as f:
        ambiguous_mapping = json.load(f)
    
    # Ensure all values are uppercase
    shipping_df['city_mapping_shp'] = shipping_df['city_mapping_shp'].str.upper()
    shipping_df['ADM2_EN'] = shipping_df['city_mapping_shp'].map(ambiguous_mapping).fillna('').str.upper()
    map_df['ADM3_EN'] = map_df['ADM3_EN'].str.upper()
    map_df['ADM2_EN'] = map_df['ADM2_EN'].str.upper()

    # Merge normally where there is no ambiguity
    normal_merge = shipping_df[~shipping_df['city_mapping_shp'].isin(ambiguous_mapping.keys())].merge(
        map_df, left_on='city_mapping_shp', right_on='ADM3_EN', how='left')

    # Merge with ADM2_EN for ambiguous cases
    ambiguous_merge = shipping_df[shipping_df['city_mapping_shp'].isin(ambiguous_mapping.keys())].merge(
        map_df, left_on=['city_mapping_shp', 'ADM2_EN'], right_on=['ADM3_EN', 'ADM2_EN'], how='left')
    
    # Concatenate the results
    merged_df = pd.concat([normal_merge, ambiguous_merge], ignore_index=True)
    
    return merged_df

def is_bold(cell):
    """Check if the cell text is bold."""
    try:
        return cell.font.bold
    except AttributeError:
        return False

def preprocess_shipping_data(file_path, sheet_name):
    """
    Preprocesses an Excel sheet from shipping data, handling multi-level headers,
    detecting bold cells, reformatting specific columns, and identifying the base rows
    for each hierarchical category before forward filling, with base indicators as integers.

    Parameters:
        file_path (str): The file path of the Excel file.
        sheet_name (str): The sheet name to preprocess.

    Returns:
        pd.DataFrame: The preprocessed DataFrame with cleaned, formatted, and enhanced data.
    """
    wb = openpyxl.load_workbook(file_path, data_only=True)
    sheet = wb[sheet_name]

    data, bold_info = [], []
    for row in sheet.iter_rows(values_only=False):
        row_data, row_bold = [], []
        for cell in row:
            row_data.append(cell.value)
            row_bold.append(is_bold(cell))
        data.append(row_data)
        bold_info.append(row_bold)

    df = pd.DataFrame(data)
    headers = df.iloc[:4].fillna('').astype(str).agg(' | '.join)
    df.columns = headers
    df = df.iloc[4:].reset_index(drop=True)

    new_columns = []
    previous_category = ""
    for col in df.columns:
        if 'Total' in col:
            previous_category = col.split('|')[0].strip()
        new_col_name = ' | '.join(c.strip() for c in col.split('|') if c.strip())
        if 'Domestic' in new_col_name or 'Foreign' in new_col_name:
            new_col_name = f"{previous_category} | {new_col_name}"
        new_columns.append(new_col_name)

    df.columns = new_columns
    df.columns.values[0:3] = ['Region', 'PMO', 'Port']

    is_bold_list = [1 if bold_row[2] else 0 for bold_row in bold_info[4:len(df)+4]]
    df['is_bold'] = is_bold_list
    df['Pier/Terminal'] = df.apply(lambda row: row['Port'] if row['is_bold'] == 0 else None, axis=1)
    df.drop(columns=['is_bold'], inplace=True)
    df['Port'] = df.apply(lambda row: None if row['Pier/Terminal'] == row['Port'] else row['Port'], axis=1)

    df.dropna(how='all', axis=1, inplace=True)
    df = df.loc[~(df.map(lambda x: pd.isna(x) or str(x).strip() == '')).all(axis=1)]
    df.reset_index(drop=True, inplace=True)

    column_order = ['Region', 'PMO', 'Port', 'Pier/Terminal'] + [col for col in df.columns if col not in ['Region', 'PMO', 'Port', 'Pier/Terminal']]
    df = df[column_order]

    df['Region'] = df['Region'].ffill()
    df['PMO'] = df.groupby('Region')['PMO'].ffill()
    df['Port'] = df.groupby(['Region', 'PMO'])['Port'].ffill()
    df['Pier/Terminal'] = df.groupby(['Region', 'PMO', 'Port'])['Pier/Terminal'].ffill()

    df['is_Region_base'] = (df['Region'].notna() & df['PMO'].isna()).astype(int)
    df['is_PMO_base'] = (df['PMO'].notna() & df['Port'].isna()).astype(int)
    df['is_Port_base'] = (df['Port'].notna() & df['Pier/Terminal'].isna()).astype(int)
    df['is_Pier/Terminal_base'] = df['Pier/Terminal'].notna().astype(int)

    # Create city_mapping_shp column
    df['city_mapping_shp'] = df.apply(lambda row: map_city(row['PMO'], row['Pier/Terminal']), axis=1)

    return df

for year in tqdm(YEARS, desc="Processing Shipping data"): 
    file_path = os.path.join('..', '[DATA] Ports Files', 'Raw Files', year, 'Shipping.xlsx')
    save_path = os.path.join('..', '[DATA] Ports Files', 'Standardized', 'Shipping', f'{year} - Shipping.{SAVE_FILE_TYPE}')
    sheet_name = 'shipcalls'
    cleaned_data = preprocess_shipping_data(file_path, sheet_name)

    # May cost time
    # merged_data = merge_with_shapefile(cleaned_data, map_df)
    # merged_data.to_csv(save_path, index=False)
    cleaned_data.to_csv(save_path, index=False)
    
print("Success!")


Processing Shipping data: 100%|████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.92s/it]

Success!
CPU times: total: 18.8 s
Wall time: 19.2 s


# Processing Passengers Data

In [5]:
%%time

def is_bold(cell):
    """Check if the cell text is bold."""
    try:
        return cell.font.bold
    except AttributeError:
        return False

def fill_and_set_bases(df, bold_info, header_rows):
    """Fill hierarchical categories and set base indicators, including handling for 'Pier/Terminal'."""
    # Ensure that the first 3 columns match the following values
    df.columns.values[0:3] = ['Region', 'PMO', 'Port']

    is_bold_list = [int(bold_row[2]) for bold_row in bold_info[header_rows:len(df)+header_rows]]
    
    df['is_bold'] = is_bold_list

    # Use bold information to differentiate 'Port' and 'Pier/Terminal'
    df['Pier/Terminal'] = df.apply(lambda row: row['Port'] if not row['is_bold'] else None, axis=1)
    df['Port'] = df.apply(lambda row: row['Port'] if row['is_bold'] else None, axis=1)

    # Remove the is_bold helper column as it is no longer needed. You can uncomment this for debugging purposes
    df.drop(columns=['is_bold'], inplace=True)

    df.dropna(how='all', inplace=True)  

    df['Region'] = df['Region'].ffill()
    df['PMO'] = df.groupby('Region')['PMO'].ffill()
    df['Port'] = df.groupby(['Region', 'PMO'])['Port'].ffill()
    df['Pier/Terminal'] = df.groupby(['Region', 'PMO', 'Port'])['Pier/Terminal'].ffill()

    df['is_Region_base'] = df['Region'].notna() & df['PMO'].isna()
    df['is_PMO_base'] = df['PMO'].notna() & df['Port'].isna()
    df['is_Port_base'] = df['Port'].notna()
    df['is_Pier/Terminal_base'] = df['Pier/Terminal'].notna()

    return df

def preprocess_passenger_data(file_path, sheet_name='passengers'):
    """
    Preprocesses an Excel sheet from passenger data, handling multi-level headers,
    detecting bold cells, reformatting specific columns, and identifying the base rows
    for each hierarchical category before forward filling, with base indicators as integers.

    Parameters:
        file_path (str): The file path of the Excel file.
        sheet_name (str): The sheet name to preprocess.

    Returns:
        pd.DataFrame: The preprocessed DataFrame with cleaned, formatted, and enhanced data.
    """
    wb = openpyxl.load_workbook(file_path, data_only=True)
    sheet = wb[sheet_name]

    data, bold_info = [], []
    for row in sheet.iter_rows(values_only=False):
        row_data, row_bold = [], []
        for cell in row:
            row_data.append(cell.value)
            row_bold.append(is_bold(cell))
        data.append(row_data)
        bold_info.append(row_bold)

    df = pd.DataFrame(data)
    headers = df.iloc[:4].fillna('').astype(str).agg(' | '.join)
    df.columns = headers
    df = df.iloc[4:].reset_index(drop=True)

    new_columns = []
    previous_category = ""
    for col in df.columns:
        if 'Total' in col:
            previous_category = col.split('|')[0].strip()
        new_col_name = ' | '.join(c.strip() for c in col.split('|') if c.strip())
        if 'Disembarked' in new_col_name or 'Embarked' in new_col_name or 'Cruise Ships' in new_col_name:
            new_col_name = f"{previous_category} | {new_col_name}"
        new_columns.append(new_col_name)

    df.columns = new_columns
    df.columns.values[0:3] = ['Region', 'PMO', 'Port']

    df = fill_and_set_bases(df, bold_info, 4)

    # df.dropna(how='all', axis=1, inplace=True)
    df = df.loc[~(df.map(lambda x: pd.isna(x) or str(x).strip() == '')).all(axis=1)]
    df.reset_index(drop=True, inplace=True)

    column_order = ['Region', 'PMO', 'Port', 'Pier/Terminal'] + [col for col in df.columns if col not in ['Region', 'PMO', 'Port', 'Pier/Terminal']]
    df = df[column_order]

    df['city_mapping_shp'] = df.apply(lambda row: map_city(row['PMO'], row['Pier/Terminal']), axis=1)

    return df

for year in tqdm(YEARS, desc="Processing Passenger data"): 
    file_path = os.path.join('..', '[DATA] Ports Files', 'Raw Files', year, 'Passenger.xlsx')
    save_path = os.path.join('..', '[DATA] Ports Files', 'Standardized', 'Passenger', f'{year} - Passenger.{SAVE_FILE_TYPE}')
    sheet_name = 'passengers'
    cleaned_data = preprocess_passenger_data(file_path, sheet_name)

    cleaned_data.to_csv(save_path, index=False)
    
print("Success!")


Processing Passenger data: 100%|███████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.62s/it]

Success!
CPU times: total: 35.5 s
Wall time: 36.2 s


# Processing Cargo Data

In [6]:
%%time

def is_bold(cell):
    """Check if the cell text is bold."""
    try:
        return cell.font.bold
    except AttributeError:
        return False

def preprocess_cargo_data(file_path, sheet_name='cargo', header_rows=4):
    """Preprocesses an Excel sheet from cargo data."""
    wb = openpyxl.load_workbook(file_path, data_only=True)
    sheet = wb[sheet_name]

    data, bold_info = [], []
    for row in sheet.iter_rows(values_only=False):
        row_data, row_bold = [cell.value for cell in row], [is_bold(cell) for cell in row]
        data.append(row_data)
        bold_info.append(row_bold)

    df = pd.DataFrame(data)
    df.dropna(how='all', axis=1, inplace=True)  
    
    headers = df.iloc[:header_rows].fillna('').astype(str).agg(' | '.join, axis=0)
    df.columns = headers
    df = df.iloc[header_rows:].reset_index(drop=True)

    df.columns = enhance_column_names(df.columns)
      

    df = fill_and_set_bases(df, bold_info, header_rows)

    column_order = ['Region', 'PMO', 'Port', 'Pier/Terminal'] + [col for col in df.columns if col not in ['Region', 'PMO', 'Port', 'Pier/Terminal']]
    df = df[column_order]

    df['city_mapping_shp'] = df.apply(lambda row: map_city(row['PMO'], row['Pier/Terminal']), axis=1)

    return df

def enhance_column_names(columns):
    """Enhances column names to include hierarchical information correctly."""
    new_columns = []
    current_month = ""
    current_category = ""
    for col in columns:
        parts = col.split(' | ')
        if 'GRAND TOTAL' in col:
            month = parts[0]
            if any(m in month for m in ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']):
                current_month = month
                current_category = ""
        elif 'DOMESTIC' in col or 'FOREIGN' in col:
            current_category = parts[0]
        new_header = f"{current_month} | {current_category} | {' | '.join(parts[1:])}"
        new_columns.append(new_header)
    return new_columns

def fill_and_set_bases(df, bold_info, header_rows):
    """Fill hierarchical categories and set base indicators, including handling for 'Pier/Terminal'."""
    # Pls dont touch these (Although, ensure that first 3 cols match the following vals)
    df.columns.values[0:3] = ['Region', 'PMO', 'Port']

    is_bold_list = [int(bold_row[2]) for bold_row in bold_info[header_rows:len(df)+header_rows]]

    df['is_bold'] = is_bold_list

    df['Pier/Terminal'] = df.apply(lambda row: row['Port'] if not row['is_bold'] else None, axis=1)
    df['Port'] = df.apply(lambda row: row['Port'] if row['is_bold'] else None, axis=1)

    df.drop(columns=['is_bold'], inplace=True)

    df.dropna(how='all', inplace=True)  

    df['Region'] = df['Region'].ffill()
    df['PMO'] = df.groupby('Region')['PMO'].ffill()
    df['Port'] = df.groupby(['Region', 'PMO'])['Port'].ffill()
    df['Pier/Terminal'] = df.groupby(['Region', 'PMO', 'Port'])['Pier/Terminal'].ffill()

    df['is_Region_base'] = df['Region'].notna() & df['PMO'].isna()
    df['is_PMO_base'] = df['PMO'].notna() & df['Port'].isna()
    df['is_Port_base'] = df['Port'].notna()
    df['is_Pier/Terminal_base'] = df['Pier/Terminal'].notna()

    return df


for year in tqdm(YEARS, desc="Processing Cargo data"):
    file_path = os.path.join('..', '[DATA] Ports Files', 'Raw Files', year, 'Cargo.xlsx')
    save_path = os.path.join('..', '[DATA] Ports Files', 'Standardized', 'Cargo', f'{year} - Cargo.{SAVE_FILE_TYPE}')
    sheet_name = 'cargo'
    cleaned_data = preprocess_cargo_data(file_path, sheet_name, 4)
    
    cleaned_data.to_csv(save_path, index=False)

print("Success!")

Processing Cargo data: 100%|███████████████████████████████████████████████████████████| 10/10 [00:47<00:00,  4.72s/it]

Success!
CPU times: total: 45.8 s
Wall time: 47.2 s


# Container

In [7]:
%%time 

def is_bold(cell):
    """Check if the cell text is bold."""
    try:
        return cell.font.bold
    except AttributeError:
        return False

def preprocess_container_data(file_path, sheet_name='container', header_rows=4):
    """Preprocesses an Excel sheet from container data."""
    wb = openpyxl.load_workbook(file_path, data_only=True)
    sheet = wb[sheet_name]

    data, bold_info = [], []
    for row in sheet.iter_rows(values_only=False):
        row_data, row_bold = [cell.value for cell in row], [is_bold(cell) for cell in row]
        data.append(row_data)
        bold_info.append(row_bold)

    df = pd.DataFrame(data)
    df.dropna(how='all', axis=1, inplace=True)  
    
    headers = df.iloc[:header_rows].fillna('').astype(str).agg(' | '.join, axis=0)
    df.columns = headers
    df = df.iloc[header_rows:].reset_index(drop=True)

    df.columns = enhance_column_names(df.columns)
    # df.dropna(how='all', inplace=True)  

    df = fill_and_set_bases(df, bold_info, header_rows)

    column_order = ['Region', 'PMO', 'Port', 'Pier/Terminal'] + [col for col in df.columns if col not in ['Region', 'PMO', 'Port', 'Pier/Terminal']]
    df = df[column_order]

    df['city_mapping_shp'] = df.apply(lambda row: map_city(row['PMO'], row['Pier/Terminal']), axis=1)

    return df

def enhance_column_names(columns):
    """Enhances column names to include hierarchical information correctly."""
    new_columns = []
    current_month = ""
    current_category = ""
    for col in columns:
        parts = col.split(' | ')
        if 'GRAND TOTAL' in col:
            month = parts[0]
            if any(m in month for m in ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']):
                current_month = month
                current_category = ""
        elif 'DOMESTIC' in col or 'FOREIGN' in col:
            current_category = parts[0]
        new_header = f"{current_month} | {current_category} | {' | '.join(parts[1:])}"
        new_columns.append(new_header)
    return new_columns

def fill_and_set_bases(df, bold_info, header_rows):
    """Fill hierarchical categories and set base indicators, including handling for 'Pier/Terminal'."""
    # Pls dont touch these (Although, ensure that first 3 cols match the following vals)
    df.columns.values[0:3] = ['Region', 'PMO', 'Port']

    is_bold_list = [int(bold_row[2]) for bold_row in bold_info[header_rows:len(df)+header_rows]]

    # Shift is_bold list up by one row
    # if len(is_bold_list) > 1:
    #     is_bold_list = is_bold_list[1:] + [0]
    
    df['is_bold'] = is_bold_list

    df['Pier/Terminal'] = df.apply(lambda row: row['Port'] if not row['is_bold'] else None, axis=1)
    df['Port'] = df.apply(lambda row: row['Port'] if row['is_bold'] else None, axis=1)

    df.drop(columns=['is_bold'], inplace=True)
    df.dropna(how='all', inplace=True)  

    df['Region'] = df['Region'].ffill()
    df['PMO'] = df.groupby('Region')['PMO'].ffill()
    df['Port'] = df.groupby(['Region', 'PMO'])['Port'].ffill()
    df['Pier/Terminal'] = df.groupby(['Region', 'PMO', 'Port'])['Pier/Terminal'].ffill()

    df['is_Region_base'] = df['Region'].notna() & df['PMO'].isna()
    df['is_PMO_base'] = df['PMO'].notna() & df['Port'].isna()
    df['is_Port_base'] = df['Port'].notna()
    df['is_Pier/Terminal_base'] = df['Pier/Terminal'].notna()

    return df

file_name = 'Container'

for year in tqdm(YEARS, desc=f"Processing {file_name} data"):
    file_path = os.path.join('..', '[DATA] Ports Files', 'Raw Files', year, f'{file_name}.xlsx')
    save_path = os.path.join('..', '[DATA] Ports Files', 'Standardized', file_name, f'{year} - {file_name}.{SAVE_FILE_TYPE}')
    sheet_name = 'teu'
    cleaned_data = preprocess_container_data(file_path, sheet_name, 4)
    cleaned_data.to_csv(save_path, index=False)

print("Success!")

Processing Container data: 100%|███████████████████████████████████████████████████████| 10/10 [00:45<00:00,  4.54s/it]

Success!
CPU times: total: 44.2 s
Wall time: 45.4 s


# RORO

In [8]:
%%time

def is_bold(cell):
    """Check if the cell text is bold."""
    try:
        return cell.font.bold
    except AttributeError:
        return False

def preprocess_roro_data(file_path, sheet_name='roro', header_rows=4):
    """Preprocesses an Excel sheet from roro data."""
    wb = openpyxl.load_workbook(file_path, data_only=True)
    sheet = wb[sheet_name]

    data, bold_info = [], []
    for row in sheet.iter_rows(values_only=False):
        row_data, row_bold = [cell.value for cell in row], [is_bold(cell) for cell in row]
        data.append(row_data)
        bold_info.append(row_bold)

    df = pd.DataFrame(data)
    df.dropna(how='all', axis=1, inplace=True)  
    
    headers = df.iloc[:header_rows].fillna('').astype(str).agg(' | '.join, axis=0)
    df.columns = headers
    df = df.iloc[header_rows:].reset_index(drop=True)

    df.columns = enhance_column_names(df.columns)
    

    df = fill_and_set_bases(df, bold_info, header_rows)
    

    column_order = ['Region', 'PMO', 'Port', 'Pier/Terminal'] + [col for col in df.columns if col not in ['Region', 'PMO', 'Port', 'Pier/Terminal']]
    df = df[column_order]

    df['city_mapping_shp'] = df.apply(lambda row: map_city(row['PMO'], row['Pier/Terminal']), axis=1)

    return df

def enhance_column_names(columns):
    """Enhances column names to include hierarchical information, correctly handling nested categories."""
    new_columns = []
    current_month = ""
    current_direction = ""
    current_type = ""       

    for col in columns:
        parts = [part.strip() for part in col.split(' | ') if part.strip()]
        if parts:
            if any(month in parts[0] for month in ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']):
                current_month = parts[0]  
                current_direction = "" 
                current_type = ""  
            if 'Inbound' in parts or 'Outbound' in parts:
                current_direction = parts[0] 
                current_type = ""  
            if 'Type' in parts[-1]:
                current_type = parts[-1]  

            if current_type:
                new_header = f"{current_month} | {current_direction} | {current_type}"
            else:
                new_header = f"{current_month} | {current_direction} | {' | '.join(parts[1:])}"
            new_columns.append(new_header)
        else:
            new_columns.append(f"{current_month} | {current_direction} | {current_type}")

    return new_columns

def fill_and_set_bases(df, bold_info, header_rows):
    """Fill hierarchical categories and set base indicators, including handling for 'Pier/Terminal'."""
    # Pls dont touch these (Although, ensure that first 3 cols match the following vals)
    df.columns.values[0:3] = ['Region', 'PMO', 'Port']

    is_bold_list = [int(bold_row[2]) for bold_row in bold_info[header_rows:len(df)+header_rows]]
    
    df['is_bold'] = is_bold_list

    # Use bold information to differentiate 'Port' and 'Pier/Terminal'
    df['Pier/Terminal'] = df.apply(lambda row: row['Port'] if not row['is_bold'] else None, axis=1)
    df['Port'] = df.apply(lambda row: row['Port'] if row['is_bold'] else None, axis=1)

    # Remove the is_bold helper column as it is no longer needed. You can uncomment ths for dbeugging purposes
    df.drop(columns=['is_bold'], inplace=True)

    df.dropna(how='all', inplace=True)  

    df['Region'] = df['Region'].ffill()
    df['PMO'] = df.groupby('Region')['PMO'].ffill()
    df['Port'] = df.groupby(['Region', 'PMO'])['Port'].ffill()
    df['Pier/Terminal'] = df.groupby(['Region', 'PMO', 'Port'])['Pier/Terminal'].ffill()

    df['is_Region_base'] = df['Region'].notna() & df['PMO'].isna()
    df['is_PMO_base'] = df['PMO'].notna() & df['Port'].isna()
    df['is_Port_base'] = df['Port'].notna()
    df['is_Pier/Terminal_base'] = df['Pier/Terminal'].notna()

    return df

file_name = 'RORO'

for year in tqdm(YEARS[1:], desc=f"Processing {file_name} data"):
    file_path = os.path.join('..', '[DATA] Ports Files', 'Raw Files', year, f'{file_name}.xlsx')
    save_path = os.path.join('..', '[DATA] Ports Files', 'Standardized', file_name, f'{year} - {file_name}.{SAVE_FILE_TYPE}')
    sheet_name = 'roro'
    cleaned_data = preprocess_roro_data(file_path, sheet_name, 4) 
    cleaned_data.to_csv(save_path, index=False)

print("Success!")


Processing RORO data: 100%|██████████████████████████████████████████████████████████████| 9/9 [00:48<00:00,  5.40s/it]

Success!
CPU times: total: 47.2 s
Wall time: 48.6 s
